In [ ]:
##Data Ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_documents=loader.load()
text_documents

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [ ]:
!pip install bs4

In [ ]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()
text_documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("attention.pdf")
docs=loader.load()

In [ ]:
docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]


In [ ]:
documents

In [ ]:
##Vector Embeddings and Vector store
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(documents[:20],OpenAIEmbeddings())
db



In [ ]:
query="What is attention is all you need"
res=db.similarity_search(query)
res[0].page_content

In [ ]:
##faiss vectordb
from langchain_community.vectorstores import FAISS
db1=FAISS.from_documents(documents[:20],OpenAIEmbeddings())
query="What is attention is all you need"
res=db1.similarity_search(query)
res[0].page_content

In [ ]:
from langchain_community.llms import ollama
llm=ollama(model="llama2")
llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """
    Answer the following question based on the provided context.
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer heplful,
    <context>
    {context}
    </context>
    Question:{input}
    """
)

In [ ]:
##Chain Introduction
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

In [ ]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [ ]:
response['answer']